In [1]:
import pandas
import numpy
import math


In [2]:
#Parse covariance matrix data
cov1 = pandas.read_csv('./dataset/DS2_Cov1.txt', header=None)
cov1.drop([20], axis=1, inplace=True)
cov1 = cov1.values

cov2 = pandas.read_csv('./dataset/DS2_Cov2.txt', header=None)
cov2.drop([20], axis=1, inplace=True)
cov2 = cov2.values

cov3 = pandas.read_csv('./dataset/DS2_Cov3.txt', header=None)
cov3.drop([20], axis=1, inplace=True)
cov3 = cov3.values

#Parse the mean vectors
c1_m1 = numpy.genfromtxt('./dataset/DS2_c1_m1.txt', delimiter=',')[:-1]
c1_m2 = numpy.genfromtxt('./dataset/DS2_c1_m2.txt', delimiter=',')[:-1]
c1_m3 = numpy.genfromtxt('./dataset/DS2_c1_m3.txt', delimiter=',')[:-1]

c2_m1 = numpy.genfromtxt('./dataset/DS2_c2_m1.txt', delimiter=',')[:-1]
c2_m2 = numpy.genfromtxt('./dataset/DS2_c2_m2.txt', delimiter=',')[:-1]
c2_m3 = numpy.genfromtxt('./dataset/DS2_c2_m3.txt', delimiter=',')[:-1]

c1_m = [c1_m1, c1_m2, c1_m3]
c2_m = [c2_m1, c2_m2, c2_m3]
cov = [cov1, cov2, cov3]


In [3]:
# 1) generate 2000 examples and split data
weight = [0.1, 0.42, 0.48]

ds1 = []
ds0 = []

for i in range(len(weight)):
    for j in range(int(2000 * weight[i])):
        ds1.append(numpy.random.multivariate_normal(c1_m[i], cov[i]))
        ds0.append(numpy.random.multivariate_normal(c1_m[i], cov[i]))


ds1 = pandas.DataFrame(ds1)
ds0 = pandas.DataFrame(ds0)
ds0[20] = 0
ds1[20] = 1

msk = numpy.random.rand(len(ds1)) <= 0.7
train1 = ds1[msk]
test1 = ds1[~msk]

msk = numpy.random.rand(len(ds0)) <= 0.7
train0 = ds0[msk]
test0 = ds0[~msk]

train = pandas.concat([train0, train1], ignore_index=True)
test = pandas.concat([test0, test1], ignore_index=True)

train1.to_csv('DS2_train1.csv')
test1.to_csv('DS2_test1.csv')
train0.to_csv('DS2_train0.csv')
test0.to_csv('DS2_test0.csv')

test.to_csv('DS2_test.csv')
train.to_csv('DS2_train.csv')

len0 = len(train0)
len1 = len(train1)

# drop the output column
train0 = train[train[20] == 0]
train0 = train0[train0.columns.difference([20])]

train1 = train[train[20] == 1]
train1 = train1[train1.columns.difference([20])]

test_output = test[20]
test.drop([20], axis=1, inplace=True)

train_output = train[20]
train.drop([20], axis=1, inplace=True)

In [4]:
# estimated max probability
print("\n\nPart 1 - LDA \n")
prob_0 = float(len0) / float(len0 + len1)
prob_1 = 1.0 - prob_0


mean_0 = numpy.array(train0.mean())
mean_1 = numpy.array(train1.mean())

diff_0 = numpy.array(train0 - mean_0)
diff_1 = numpy.array(train1 - mean_1)

coeff_0 = numpy.matmul(diff_0.T, diff_0)
coeff_1 = numpy.matmul(diff_1.T, diff_1)

coeff = (coeff_0 + coeff_1) / float(len0 + len1)

covm_0 = numpy.matmul(numpy.linalg.pinv(coeff), mean_0)
mcovm_0 = numpy.matmul(mean_0.T, covm_0)

covm_1 = numpy.matmul(numpy.linalg.pinv(coeff), mean_1)
mcovm_1 = numpy.matmul(mean_1.T, covm_1)

w0 = math.log(prob_0) - math.log(prob_1) - 1 / 2 * (mcovm_0 - mcovm_1)
w1 = numpy.matmul(numpy.linalg.pinv(coeff), mean_0 - mean_1)


print("w0: ", w0)
print("w1: " + str([i for i in w1]) + "\n")

pred = numpy.matmul(test, w1) + w0

# replace dec_bound to 0 and 1
pred[pred > 0] = 0
pred[pred < 0] = 1

error_matrix = pred - test_output

elem, cnt = numpy.unique(error_matrix, return_counts=True)

print(numpy.asarray([elem, cnt]))

'''
1 => false positive
0 => correct prediction
-1 => false negative
'''

fp = cnt[2]
fn = cnt[0]


tp = numpy.sum(pred.dot(test_output))
print("True positive: " + str(tp))

pred = pred - 1
test_output_temp = test_output - 1
tn = numpy.sum(pred.dot(test_output_temp))
print("True negative: " + str(tn))

precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tn + tp) / (tn + tp + fn + fp)

F = 2 * precision * recall / (precision + recall)

print("\nF measure: " + str(F))
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))




Part 1 - LDA 

w0:  -0.0279488037596
w1: [-0.040210744774170547, -0.00173584274637828, 0.051531460986256022, -0.0062934314688421929, -0.021509305937748233, 0.031543193370196218, -0.051978360151621938, 0.025362995756740891, -0.022105758086344216, 0.018874725247577537, 0.052170047457334334, 0.0053750125135964173, 0.0072283688545865463, -0.0036520459414803932, -0.008406153162515459, 0.063305120942572207, -0.0031908746996294566, -0.024487927370320933, -0.077634901759780731, 0.0060027925674444246]

[[  -1.    0.    1.]
 [ 221.  589.  365.]]
True positive: 348.0
True negative: 241.0

F measure: 0.542901716069
Accuracy: 0.501276595745
Precision: 0.488078541374
Recall: 0.611599297012


In [5]:
print("\n\nPart 2 - K NN\n")


res_step = []
accuracy_step = []


for k in range(1, 50):
    # metrics = [tp, tn, fp, fn]
    metrics = [0, 0, 0, 0]
    for i in range(len(test)):
        dist = abs(train.sub(numpy.array(numpy.array(test.loc[[i], :])[0])))
        dist = numpy.array(numpy.power(dist, 2).sum(axis=1))

        # select the k training examples closest to the test example
        ind = numpy.argpartition(dist, k)[:k]

        # predict value of test output using the average of the k training outputs
        dec = 1 if (numpy.array([train_output[i] for i in ind]).mean() > 0.5) else 0

        #update metrics
        if dec == test_output[i] == 1:
            metrics[0] += 1
        elif dec == test_output[i] == 0:
            metrics[1] += 1
        elif dec != test_output[i] == 1:
            metrics[2] += 1
        else:
            metrics[3] += 1

    res_step.append(metrics)
    accuracy_step.append((metrics[0] + metrics[1]) / numpy.sum(metrics))
print(accuracy_step)

print("\nBest k = " + str(accuracy_step.index(max(accuracy_step)) + 1))

metrics = res_step[accuracy_step.index(max(accuracy_step))]
tp = metrics[0]
tn = metrics[1]
fp = metrics[2]
fn = metrics[3]

precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tn + tp) / (tn + tp + fn + fp)

F = 2 * precision * recall / (precision + recall)

print("\nAccuracy = " + str(max(accuracy_step)))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F - measure = " + str(F))






Part 2 - K NN

[0.49021276595744678, 0.49872340425531914, 0.48085106382978721, 0.50382978723404259, 0.47999999999999998, 0.49531914893617024, 0.48765957446808511, 0.465531914893617, 0.48170212765957449, 0.47914893617021276, 0.47914893617021276, 0.48255319148936171, 0.49276595744680851, 0.49361702127659574, 0.48170212765957449, 0.48085106382978721, 0.48851063829787233, 0.49191489361702129, 0.49787234042553191, 0.49446808510638296, 0.50382978723404259, 0.50212765957446803, 0.50553191489361704, 0.50042553191489358, 0.51234042553191494, 0.50212765957446803, 0.50212765957446803, 0.50212765957446803, 0.49617021276595746, 0.49021276595744678, 0.49361702127659574, 0.49276595744680851, 0.49446808510638296, 0.47319148936170213, 0.49361702127659574, 0.47234042553191491, 0.49106382978723406, 0.48340425531914893, 0.49361702127659574, 0.48170212765957449, 0.49276595744680851, 0.48085106382978721, 0.49021276595744678, 0.49021276595744678, 0.49106382978723406, 0.48510638297872338, 0.4791489361702127